In [7]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import json
from gensim.utils import deaccent

unable to import 'smart_open.gcs', disabling that module


In [67]:
class DownloaderBezRealitky():
    def __init__(self):
        '''
        For the bezrealitky search, you need to iterate over search pages. Self.page_bezrealitky stores the maximum amount
        of pages and then via self.link a list of all pages from search is created in self.hrefs_bezrealitky.
        '''
        self.link = 'https://www.bezrealitky.cz/vypis/nabidka-pronajem/byt/praha?_token=pr1lf-vKwDFfmFbICiz2PfC-Zdwq-2JolXi4MeMHsrw&page=1'
        self.request = requests.get(self.link)
        self.request.encoding='UTF-8'
        self.soup = BeautifulSoup(self.request.text,'lxml')
        
        self.page_bezrealitky = int(self.soup.findAll('a',{'class':'page-link pagination__page'})[-2].text)
        self.hrefs_bezrealitky = ['https://www.bezrealitky.cz/vypis/nabidka-pronajem/byt/praha?_token=pr1lf-vKwDFfmFbICiz2PfC-Zdwq-2JolXi4MeMHsrw&page=' 
                                  + str(i) for i in range(1,self.page_bezrealitky)]
        self.soups = []
        
    def get_soups(self):
        '''
        This method iterates over all search pages, converts them into a BeautifulSoup object and stores them in a txt file as 
        strings outside of this script. BREAKHERE is used to distinguish between objects. 
        '''
        for link in self.hrefs_bezrealitky:
            sleep(1)
            self.link = link
            self.request = requests.get(self.link)
            self.request.encoding='UTF-8'
            self.soups.append(BeautifulSoup(self.request.text,'lxml'))
        with open('bezrealitky_links.txt', 'w') as f:
            for s in self.soups:
                f.write(str(deaccent(s).encode("utf-8")) + 'BREAKHERE')
            f.close
        
    def get_data(self):
        '''
        Main method to obtain and transform the data. HTMLs are read from the txt file and stored in a list (soup_list) 
        within this script. Next, the method iterates over the list, converts the strings in txt file into a BeautifulSoup
        object and parses the html for relevant data. At the end, a nested dictionary (dicts) is created and stored
        as a json file outside of this script.
        '''
        with open('bezrealitky_links.txt', 'r') as f:
            content = f.read()
        soup_list = content.split('BREAKHERE')
        dicts = {}
        counter = 0
        for soup in soup_list:
            descrips = [] #empty list for apartment values
            values = [] #empty list for apartment prices
            vals = BeautifulSoup(soup,'lxml').findAll('strong', {'class':'product__value'}) #parsing for apartment values
            ##vals = soup.findAll('strong', {'class':'product__value'})
            for vl in vals:
                values.append(vl.text.strip())
            #img = soup.findAll('img')
            img = BeautifulSoup(soup,'lxml').findAll('img') #parsing for apartment info (street, city, size..)
            for i in img:
                if 'Pronajem' and 'obr. c. 1' in i['alt']: #info present at all pictures, let's take info from the first one
                        info = i['alt'].split(',')[0:4] #info separated by comma, split into a list
                        if 'Praha' == info[-1].strip(): #if street non present, insert a NaN instead
                            info.insert(2, 'NaN')
                            del info[-1]
                            m = info[1].split(' ')
                            info[1] = m[1]
                            descrips.append(info)
                        else:
                            m = info[1].split(' ')
                            info[1] = m[1]
                            descrips.append(info)
            count = 0
            for pp in values: #append apartment prices to info about apartments in list descrips
                try:
                    descrips[count].append(pp)
                    descrips[count][0] = descrips[count][0][-4:].strip()
                    count += 1
                except IndexError:
                    count += 1
                    continue
            for item in descrips:
                try:
                    if '+' in item[4]: #prices often written as '19000 Kč + 4000Kč' so we need to split it
                        prices = item.pop(4).split('+')
                        item.append(re.sub("[^0-9]", "", prices[0])) #keep only numeric characters, i.e. price
                        item.append(re.sub("[^0-9]", "", prices[1]))
                    else:
                        prices = [item.pop(4), '0'] #if only '19000 Kč', insert 0 as price for utilities not specified
                        item.append(re.sub("[^0-9]", "", prices[0]))
                        item.append(re.sub("[^0-9]", "", prices[1]))
                except IndexError:
                    continue
            for item in descrips: #store apartment info, price into a dictionary and index by counter
                try:
                    dict = {}
                    dict['Size'] = item[0]
                    dict['m2'] = re.sub("[^0-9]", "", item[1]) #keep only size, i.e. numeric characters
                    dict['Street'] = deaccent(item[2]) #deaccent to provent potential errors
                    dict['District'] = deaccent(item[3])
                    dict['Base Price'] = int(item[4])
                    dict['Utilities Price'] = int(item[5])
                    dict['Total Price'] = int(item[4]) + int(item[5])
                    dicts[counter] = dict
                    counter += 1
                except IndexError:
                    counter +=1
                    continue
            print('Done loop number ' + str(counter) + '. Printing descrips.')
            print(descrips)
        with open('bezrealitky.json', 'w') as write_file: #store data into a json file
            json.dump(dicts, write_file, indent = 4)

        
            
        

In [69]:
a = DownloaderBezRealitky()

In [65]:
a.get_soups()

In [ ]:
a.get_data()

In [172]:
class DownloaderReality():
    def __init__(self):
        '''
        For the reality search, you need to iterate over search pages. Self.page_reality stores the maximum amount
        of pages and then via self.link a list of all pages from search is created in self.hrefs_reality.
        '''
        self.link = 'https://reality.idnes.cz/s/pronajem/byty/praha/?page=1'
        self.request = requests.get(self.link)
        self.request.encoding='UTF-8'
        self.soup = BeautifulSoup(self.request.text,'lxml')
        
        self.page_reality = int(self.soup.findAll('a',{'class':'btn btn--border paging__item'})[-1].text) - 1
        self.hrefs_reality = ['https://reality.idnes.cz/s/pronajem/byty/praha/?page=' 
                                  + str(i) for i in range(1,self.page_reality)]
        self.soups = []
        
    def get_soups(self):
        '''
        This method iterates over all search pages, converts them into a BeautifulSoup object and stores them in a txt file as 
        strings outside of this script. BREAKHERE is used to distinguish between objects. 
        '''
        for link in self.hrefs_reality[0:2]:
            sleep(1)
            self.link = link
            self.request = requests.get(self.link)
            self.request.encoding='UTF-8'
            self.soups.append(BeautifulSoup(self.request.text,'lxml'))
            print('Page saved.')
        with open('reality_idnes_links.txt', 'w') as f:
            for s in self.soups:
                f.write(str(deaccent(s).encode("utf-8")) + 'BREAKHERE')
            f.close
    def get_data(self):
        '''
        Main method to obtain and transform the data. HTMLs are read from the txt file and stored in a list (soup_list) 
        within this script. Next, the method iterates over the list, converts the strings in txt file into a BeautifulSoup
        object and parses the html for relevant data. At the end, a nested dictionary (dicts) is created and stored
        as a json file outside of this script.
        '''
        with open('reality_idnes_links.txt', 'r') as f:
            content = f.read()
        soup_list = content.split('BREAKHERE')
        dicts = {}
        counter = 0
        for soup in soup_list[0:2]:
            descrips = [] #empty list for apartment values
            values = [] #empty list for apartment prices
            vals = BeautifulSoup(soup,'lxml').findAll('p', {'class':'c-list-products__price'}) #parsing for apartment values
            for vl in vals:
                values.append(re.sub("[^0-9]", "",vl.find('strong').text))
            info = BeautifulSoup(soup,'lxml').findAll('p', {'class':'c-list-products__info'})
            for i in info:
                if 'Komercni sdeleni' in i.text:
                    continue
                else:
                    temp_info = str(i.text)
                    temp_info = re.sub(r'^(?:\\n)+','', temp_info).strip()[:-2]
                    temp_info = temp_info.strip().split(',')
                    temp_info = [i.strip() for i in temp_info]
                    print(temp_info)
                
        
            '''
            for item in descrips: #store apartment info, price into a dictionary and index by counter
                try:
                    dict = {}
                    dict['Size'] = item[0]
                    dict['m2'] = re.sub("[^0-9]", "", item[1]) #keep only size, i.e. numeric characters
                    dict['Street'] = deaccent(item[2]) #deaccent to provent potential errors
                    dict['District'] = deaccent(item[3])
                    dict['Base Price'] = int(item[4])
                    dict['Utilities Price'] = int(item[5])
                    dict['Total Price'] = int(item[4]) + int(item[5])
                    dicts[counter] = dict
                    counter += 1
                except IndexError:
                    counter +=1
                    continue
            print('Done loop number ' + str(counter) + '. Printing descrips.')
            print(descrips)
        with open('bezrealitky.json', 'w') as write_file: #store data into a json file
            json.dump(dicts, write_file, indent = 4)
            '''

In [173]:
b = DownloaderReality()

In [151]:
b.get_soups()

Page saved.
Page saved.


In [174]:
b.get_data()

['U skladu', 'Praha 4 - Komorany']
['Musilkova', 'Praha 5 - Kosire']
['Milevska', 'Praha 4 - Krc']
['Duskova', 'Praha 5 - Smichov']
['Taborska', 'Praha 4 - Nusle', 'okres Praha']
['Truhlarska', 'Praha 1 - Nove Mesto']
['Spalena', 'Praha 1 - Nove Mesto', 'okres Praha']
['Tesnov', 'Praha 1 - Nove Mesto', 'okres Praha']
['Nuselska', 'Praha 4 - Nusle', 'okres Praha']
['Jankovcova', 'Praha 7 - Holesovice']
['Opatovicka', 'Praha 1 - Nove Mesto', 'okres Praha']
['Vinohradska', 'Praha 3 - Vinohrady', 'okres Praha']
['Balbinova', 'Praha 2 - Vinohrady', 'okres Praha']
['Na Kozacce', 'Praha 2 - Vinohrady']
['Zverinova', 'Praha 3 - Strasnice', 'okres Praha']
['Reznicka', 'Praha 1 - Nove Mesto', 'okres Praha']
['Krynicka', 'Praha 8 - Troja', 'okres Praha']
['Kalisnicka', 'Praha 3 - Zizkov', 'okres Praha']
['Hornomecholupska', 'Praha 10 - Horni Mecholupy', 'okres Praha']
['Mrkvickova', 'Praha 6 - Repy']
['Korunni', 'Praha 10 - Vinohrady']
['Chalabalova', 'Praha 5 - Stodulky']
['Smetanovo nabrezi', '